In [1]:
print('hello')

hello


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/ml_plus/data')

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
movies =  pd.read_csv('영화 & TV 데이터.csv', index_col= 0, encoding='CP949')

In [11]:
movies = movies.dropna()

In [37]:
movies = movies.reset_index()

In [40]:
movies = movies.drop(columns='index')

In [41]:
movies.head(2)

,contentId,title,genre,content,imgUrl,contentRating,createdYear,ratingCount,type
0,315162,장화신은 고양이: 끝내주는 모험,"'애니메이션', '모험', '코미디', '가족'",아홉 개의 목숨 중 단 하나의 목숨만 남은 장화신은 고양이. 마지막 남은 목숨을 지...,https://image.tmdb.org/t/p/w342/rKgvctIuPXyuqO...,8.4,2022-12-07,4715,movie
1,505642,블랙 팬서: 와칸다 포에버,"'액션', '모험', 'SF'",국왕이자 블랙 팬서인 티찰라의 죽음 이후 수많은 강대국으로부터 위협을 받게 된 와칸...,https://image.tmdb.org/t/p/w342/3PCRWLeqp5y20k...,7.3,2022-11-09,4193,movie


In [42]:
# 장르 데이터 전처리
movies['genre'] = movies['genre'].str.replace('|', ' ')

# TF-IDF 벡터화
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['genre'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 추천 함수
def recommend_movies(titles, cosine_sim=cosine_sim):
    indices = pd.Series(movies.index, index=movies['title'])
    idx_list = []
    for title in titles:
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        movie_indices = [i[0] for i in sim_scores]
        idx_list.extend(movie_indices)
    idx_list = list(set(idx_list))
    df_rec = movies.iloc[idx_list][['contentId', 'title', 'genre']].reset_index(drop=True)
    df_rec = df_rec[~df_rec['title'].isin(titles)].head(10)
    return df_rec

<ipython-input-42-78310bc64a97>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies['genre'] = movies['genre'].str.replace('|', ' ')


In [44]:
recommend_movies(['문워커','써클: 이어진 두 세계','태양의 서커스: 월드 어웨이'])

,contentId,title,genre
1,17917,올리버,"'드라마', '가족', '음악'"
2,193613,메탈리카 스루 더 네버,'음악'
3,553512,번 더 스테이지: 더 무비,'음악'
4,83300,디아블레로: 악마 사냥꾼,"'SF&판타지', '미스터리'"
5,3868,로스트 룸,"'미스터리', 'SF&판타지'"
6,244786,위플래쉬,"'드라마', '음악'"
7,568332,테일러 스위프트의 레퓨테이션 스타디움 투어,'음악'
8,555604,기예르모 델토로의 피노키오,"'애니메이션', '판타지', '드라마', '음악'"
9,1622,수퍼내추럴,"'드라마', '미스터리', 'SF&판타지'"
10,1041513,엔칸토 할리우드 볼,'음악'


In [45]:
from joblib import dump, load

# 추천 함수 저장
dump(recommend_movies, 'recommend_movies.joblib')

['recommend_movies.joblib']

In [ ]:
# 추천 함수 불러오기
# recommend_movies = load('recommend_movies.joblib')

In [ ]:
# 불러오기 후 사용법법
# titles = ['문워커', '써클: 이어진 두 세계']
# recommend_movies(titles)